In [2]:
import os

CLEARML_API_ACCESS_KEY = os.getenv("CLEARML_API_ACCESS_KEY")
CLEARML_API_SECRET_KEY = os.getenv("CLEARML_API_SECRET_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")

missing_vars = []
if not CLEARML_API_ACCESS_KEY:
  missing_vars.append("CLEARML_API_ACCESS_KEY")
if not CLEARML_API_SECRET_KEY:
  missing_vars.append("CLEARML_API_SECRET_KEY")
if not OPENAI_API_KEY:
  missing_vars.append("OPENAI_API_KEY")
if not SERPAPI_API_KEY:
  missing_vars.append("SERPAPI_API_KEY")

if missing_vars:
  print(f"The following environment variables are missing: {', '.join(missing_vars)}")
else:
  print("All environment variables are available.")


All environment variables are available.



Scenario 2: Creating an agent with tools

To show a more advanced workflow, let's create an agent with access to tools. The way ClearML tracks the results is not different though, only the table will look slightly different as there are other types of actions taken when compared to the earlier, simpler example.

You can now also see the use of the finish=True keyword, which will fully close the ClearML Task, instead of just resetting the parameters and prompts for a new conversation.

In [3]:
from langchain.callbacks import ClearMLCallbackHandler

from datetime import datetime
from langchain.callbacks import StdOutCallbackHandler
from langchain.llms import OpenAI

# Setup and use the ClearML Callback
clearml_callback = ClearMLCallbackHandler(
    task_type="inference",
    project_name="langchain_callback_demo",
    task_name="llm",
    tags=["test"],
    # Change the following parameters based on the amount of detail you want tracked
    visualize=True,
    complexity_metrics=True,
    stream_logs=True,
)
callbacks = [StdOutCallbackHandler(), clearml_callback]
# Get the OpenAI model ready to go
llm = OpenAI(temperature=0, callbacks=callbacks)

ClearML Task: created new task id=81e6fadad4e04e0d80ec0db0f51f9656
2023-09-16 23:01:35,289 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/af7507b5860447cd89ecf7d1ae3166c8/experiments/81e6fadad4e04e0d80ec0db0f51f9656/output/log
The clearml callback is currently in beta and is subject to change based on updates to `langchain`. Please report any issues to https://github.com/allegroai/clearml/issues with the tag `langchain`.


https://api.python.langchain.com/en/latest/agents/langchain.agents.initialize.initialize_agent.html

https://api.python.langchain.com/en/latest/agents/langchain.agents.load_tools.load_tools.html

https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_types.AgentType.html


In [4]:

from langchain.agents import initialize_agent, load_tools
from langchain.agents import AgentType

# SCENARIO 2 - Agent with Tools
tools = load_tools(["serpapi", "llm-math"], llm=llm, callbacks=callbacks)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    callbacks=callbacks,
)
agent.run("Who is the wife of the person who sang summer of 69?")
clearml_callback.flush_tracker(
    langchain_asset=agent, name="Agent with Tools", finish=True
)

ValidationError: 1 validation error for SerpAPIWrapper
__root__
  Could not import serpapi python package. Please install it with `pip install google-search-results`. (type=value_error)

ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start
